<a href="https://colab.research.google.com/github/hopefulhazel/programacion/blob/master/Copia_de_Narrative_clip_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training

In [0]:
# Cargamos la versión 2.x de TensorFlow
try:
  # %tensorflow_version sólo existe en Colab
  %tensorflow_version 2.x
except Exception:
  pass


from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
from google.colab import drive

drive.mount('/content/drive')


In [0]:
!wget https://ub.panalsoft.com/labeling/public/api/photos/download-csv -O /content/drive/My\ Drive/database/database.csv


In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/content/drive/My Drive/database/photos/_g4/42770/conversions/172445-thumb.jpg')
imgplot = plt.imshow(img)
plt.show()

In [0]:
# importing pandas package
import pandas as pd
 
# making data frame from csv file
df = pd.read_csv("/content/drive/My Drive/database/database.csv")
print(df.head(5))

df['path'] = "/content/drive/My Drive/database/photos/" + "_g" + df.media_id.map(str).map(lambda x: x[:1]) + "/" + df.media_id.map(str) + "/conversions/" + df.filename.map(lambda x: x[:-4]) + "-thumb.jpg"
df = df[['path', 'label']]
df['path'] = df['path'].astype(str)

# remove labels with a lot of rows
df_filter = df['label'] != 'Using a Computer'
df = df[df_filter]

# ramdom sorting the dataframe
df = df.sample(frac=1)

TO_TEST = 100
df_test = df.head(TO_TEST)

#df = df.tail(len(df) - TO_TEST)
df = df.tail(10000 - TO_TEST)

print(df.head(5))


In [0]:
df.label.value_counts().plot(kind="bar", figsize=(15,8))

In [0]:
# Importamos TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

# Importamos algunas librerías de apoyo como numpy y matplot
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import pathlib

EPOCHS = 10
BATCH_SIZE = 8
LR = 2e-5
IMGW = 299
IMGH = 299
IMGN = 255

#datagen = ImageDataGenerator(rescale = 1.0 / IMGN)

datagen = ImageDataGenerator(
    rescale = 1. / IMGN,
    #rotation_range = 90,
    #width_shift_range = 0.2,
    #height_shift_range = 0.2,
    #shear_range = 0.2,
    #zoom_range = 0.2,
    #horizontal_flip = True,
    #fill_mode = 'nearest',
    validation_split=0.2
)


train_images = datagen.flow_from_dataframe(
    dataframe=df, 
    directory=None, 
    x_col="path", 
    y_col="label", 
    class_mode="categorical", 
    target_size = (IMGW, IMGH),
    batch_size = BATCH_SIZE,
    subset="training"
)

validation_images = datagen.flow_from_dataframe(
    dataframe=df, 
    directory=None, 
    x_col="path", 
    y_col="label", 
    class_mode="categorical", 
    target_size = (IMGW, IMGH),
    batch_size = BATCH_SIZE,
    subset="validation"
)

CLASSES = 24


In [0]:
CLASSES = 23

In [0]:
from tensorflow.keras import applications, optimizers
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

base_model = applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(IMGW, IMGH, 3))

model = tf.keras.Sequential()

model.add(base_model)

TRAINABLE_LAYERS = 1

for layer in base_model.layers[:-TRAINABLE_LAYERS]:
    layer.trainable=False

for layer in base_model.layers[-TRAINABLE_LAYERS:]:
    layer.trainable=True


model.add(layers.Flatten())
model.add(layers.Dense(CLASSES, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=LR), metrics=["accuracy"])

model.summary()


In [0]:
history_transfer = model.fit(
  train_images, 
  #batch_size = BATCH_SIZE, 
  epochs = EPOCHS, 
  #verbose = 2, 
  #callbacks = None, 
  validation_data = validation_images, 
  #class_weight = None, 
  #sample_weight=None, 
  #steps_per_epoch=None, 
  #validation_steps=None, 
  #workers=1, 
  #use_multiprocessing = True,
  shuffle = True
)



In [0]:
# Plot training & validation accuracy values
plt.plot(history_transfer.history['accuracy'])
plt.plot(history_transfer.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_transfer.history['loss'])
plt.plot(history_transfer.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [0]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict_generator(validation_images, CLASSES // BATCH_SIZE+1)
y_pred = np.argmax(Y_pred, axis=1)

print(confusion_matrix(validation_images.classes, y_pred))
#print(confusion_matrix(validation_images.classes, y_pred, df.label_name.unique()))

#print('Classification Report')
#target_names = ['Dishwashing', 'Using a Computer']
#print(classification_report(validation_images.classes, y_pred, target_names=target_names))


In [0]:
model.save('/content/drive/My Drive/database/model1.h5')

# Exportar el modelo a 'SavedModel'
#keras.experimental.export_saved_model(model, '/content/drive/My Drive/database/model1')

# Recrea exactamente el mismo modelo
#new_model = keras.experimental.load_from_saved_model('/content/drive/My Drive/database/model1')


In [0]:
new_model = keras.models.load_model('/content/drive/My Drive/database/model1.h5')

In [0]:
from keras.preprocessing.image import load_img, img_to_array

from io import BytesIO
from urllib.request import urlopen

labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())

def predict(model, image_url):
#  img = load_img(f, target_size=(hyperparams["img_width"], hyperparams["img_height"]))

  with urlopen(image_url) as url:
    url_read = BytesIO(url.read())
    img_original = load_img(url_read)
    img = load_img(url_read, target_size=(299, 299))

  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)

  plt.imshow(img_original)
  plt.show()

  img = datagen.standardize(img)

  res = model.predict(img)  
  print("predicts: " + labels[np.argmax(res)])  

  res = np.around(res[0], decimals=2)  
  print(res)
  print('--------------------')

  return res


In [0]:
predict(new_model, 'https://s3.amazonaws.com/ub.labeling/72936/conversions/114249-thumb.jpg')
predict(new_model, 'https://s3.amazonaws.com/ub.labeling/92932/conversions/145453-thumb.jpg')
predict(new_model, 'https://images.wsj.net/im-112660?width=620&size=1.5')
predict(new_model, 'https://article.images.consumerreports.org/f_auto/prod/content/dam/CRO%20Images%202019/Health/01January/CR-Health-InlineHero-Get-Biggest-Benefits-of-Walking-01-19-v2')
predict(new_model, 'https://image.shutterstock.com/image-photo/modern-business-people-walking-on-260nw-739813360.jpg')

print('')

